**Road Segmentation**

Using camvid dataset for road segmentation.



In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
import albumentations as A

import tensorflow as tf
from tensorflow import keras

#from tf_seg.data import DataLoader
from tf_seg.config import get_config
from tf_seg.data import get_data_loader,get_camvid_data_loader
from tf_seg.models import Unet
from tf_seg.losses import FocalTverskyLoss,DiceLoss
from tf_seg.metrics import DiceScore
from tf_seg.train import Trainer
from tf_seg.callbacks import get_callbacks
from tf_seg.transformers import Transformer

from tensorflow.keras.losses import binary_crossentropy,BinaryCrossentropy
from tensorflow.keras.metrics import binary_accuracy,BinaryAccuracy

Config mananagment can be two ways: 
- First, you can use python dict object to configure the data,model etc.
- Second, you can use a config file as yaml format. if you use this way, you must ```get_config``` function to get the config file and check if it is valid.


In [ ]:
# camvid data config

data_config = dict(
    name="road_segmentation",
    function_name="camvid",  # it is used camvid dataset to generate binary data
    path="../dataset/camvid",
    classes=["Road"],
    normalizing=True,
    palette=[(128, 64, 128)],
    one_hot_encoding=True,  # target output shape
    background_adding=False,  # add target background class
    image_size=(512, 512),
    batch_size=8,
    # output_type=("tf.float32", "tf.float32"),  # this is for camvid data types after data processing
    channels=(3, 3),  # it is optional
)

model_config = dict(
    # n_filters=[16, 32, 64, 128, 256],
    # n_filters=[4, 8, 12, 16,24],
    n_filters=[16, 24, 32, 64],  #
    input_shape=[data_config["image_size"][0], data_config["image_size"][1], 3],
    final_activation="sigmoid",
    activation="relu",
    backbone= "EfficientNetB0",# "ResNet50", None
    pretrained="imagenet",
    output_size=1,
)

# we will load albermentations functions manually, so there is no need to all parameters like function "path"
aug_config = dict(aug_type="albumentations")
# config file look like this

# load_style: module # {module, file} # it find automatically
# aug_type: albumentations
# train:
#    path: tf_seg.transformers.albumentations:get_train_transform
#    parameters: { image_size: [512, 512], p: 0.5 }
# val:
#    path: tf_seg.transformers.albumentations:get_test_transform
#    parameters: { image_size: [512, 512] }
# test:
#    path: #tf_seg.transformers.albumentations:get_test_transform
#    parameters: { image_size: [512, 512] }


trainer_config = dict(
    epochs=20,
    batch_size=8,
    optimizer={"name": "adam", "params": {"learning_rate": 0.001}},
    losses=["binary_crossentropy"],
    metrics=["binary_accuracy"],
    save_model=True,
    save_name="test_efficientnetb0_binary_road",
    verbose=1,
    deploy_onnx=True,
)

callbacks_config = dict(measure_total_time={"class_name": "MeasureTotalTime", "params": {}})

config = dict(data=data_config, model=model_config, aug=aug_config, trainer=trainer_config, callbacks=callbacks_config)


In [ ]:
# get_camvid_data_loader(data_config, train_data=True, val_data=True, test_data=False)
# get_data_loader(data_config, train_data=True, val_data=True, test_data=False) #or, this is a selection function


#### DataLoader

This function is support function getting a DataLoader object that is loading data from disk as tensorflow dataset. There are many special data loader functions for different dataset. Also, there is a custom data loader function for custom datasets.



In [ ]:
# there are three parts as train, val, test in camvid dataset
train_data_loader, val_data_loader, test_data_loader = get_camvid_data_loader(data_config)
# train_data_loader.load_data? # show docstring


In [ ]:
# generate dataset from data_loader object via load_data function
train_dataset = train_data_loader.load_data()
val_dataset = val_data_loader.load_data()
test_dataset = test_data_loader.load_data()


In [ ]:
for i in train_dataset.take(1):
    print(i[0].shape)
    print(i[1].shape)

In [ ]:
image = i[0][0].numpy()
image.shape

In [ ]:
plt.imshow(image)

In [ ]:
def show_data(train_dataset=None, val_dataset=None, test_dataset=None):
    "support function to show data"

    plt.figure(figsize=(15, 10))
    if train_dataset:
        for i, m in train_dataset.take(1):
            plt.subplot(2, 3, 1)
            plt.imshow(m[0])
            plt.subplot(2, 3, 4)
            plt.imshow(i[0].numpy())
    if val_dataset:
        for i, m in val_dataset.take(1):
            plt.subplot(2, 3, 2)
            plt.imshow(m[0])
            plt.subplot(2, 3, 5)
            plt.imshow(i[0].numpy())
    if test_dataset:
        for i, m in test_dataset.take(1):
            plt.subplot(2, 3, 3)
            plt.imshow(m[0])
            plt.subplot(2, 3, 6)
            plt.imshow(i[0].numpy())


show_data(train_dataset, val_dataset, test_dataset)


**With augmentation function**

This is ```albumentations``` function to augement the data.

In [ ]:
IM_SIZE = data_config["image_size"][0]


train_transforms = A.Compose(
    [
        A.Resize(IM_SIZE, IM_SIZE),
        A.OneOf(
            [
                A.RandomSizedCrop(min_max_height=(256, 256), height=IM_SIZE, width=IM_SIZE, p=0.5),
                A.CenterCrop(height=IM_SIZE, width=IM_SIZE, p=0.5),
                A.PadIfNeeded(min_height=IM_SIZE, min_width=IM_SIZE, p=0.5),
            ],
            p=1,
        ),
        A.OneOf([A.VerticalFlip(p=0.5), A.RandomRotate90(p=0.5), A.Transpose(p=0.5)]),
        A.OneOf([A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.5), A.GridDistortion(p=0.5), A.OpticalDistortion(distort_limit=2, shift_limit=0.5, p=1)], p=0.8),
    ]
)


test_transforms = A.Compose(
    [
        A.Resize(IM_SIZE, IM_SIZE),
    ]
)


```Transformer``` is a class to use albumentations fucntions or other augmentation packages with tf.data.Dataset.map function effectively.



In [ ]:
tr_transforms_object = Transformer(aug_config, "train", train_transforms)
ts_transforms_object = Transformer(aug_config, "test", test_transforms)

# get datasets with augmentation

train_dataset = train_data_loader.load_data(transform_func=tr_transforms_object)
val_dataset = val_data_loader.load_data(transform_func=ts_transforms_object)
test_dataset = test_data_loader.load_data(transform_func=ts_transforms_object)


In [ ]:
show_data(train_dataset, val_dataset, test_dataset)

#### Make Model

In [ ]:
trainer_config

In [ ]:
for i in train_dataset.take(1):
    print(i[0].shape)
    print(i[1].shape)

In [ ]:
model = Unet(**model_config).build_model()
model.summary()

In [ ]:
callbacks = get_callbacks(callbacks_config)
callbacks

In [ ]:
all_config = dict(model=model_config, data=data_config, trainer=trainer_config)
trainer = Trainer(all_config, model, train_dataset, val_dataset, callbacks)

In [ ]:
trainer.train()

In [ ]:
trainer.train(continue_training=True)

In [ ]:
for i in train_dataset.take(1):pass
pred = model.predict(i[0])

In [ ]:
for a in range(8):
    r  = pred[a]
    re = r>0.5
 
    plt.subplot(1,2,1)
    plt.imshow(i[1][a])
    plt.subplot(1,2,2)
    plt.imshow(re.astype(int))
    plt.show()